## Preparación del entorno

In [1]:
import numpy as np

import sklearn

import pandas as pd

Si el entorno está correctamente instalado, las líneas de código anteriores deben importar los paquetes sin ningún error.

Nota: para el resto de las preguntas y soluciones de código, puede ingresar más celdas si lo considera necesario.


## Carga y estudio de datos

Cargue los datos desde el archivo *adult_data.csv*. Para esto puede utilizar la librería *pandas* con su función *read_csv*.

In [2]:
data = pd.read_csv('adult_data.csv')

Imprima los nombres de las columnas (atributos).

In [3]:
data.head(0)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income


## Extracción de atributos

Separar la columna **income** en un array **y** que será utilizada como atributo clase:

In [4]:
y = data['income']

Eliminar la columna **fnlwgt** ya que no aporta a la solución del problema. También eliminar la columna **education-num** ya que duplica la información de la columna 'education'. Por último, eliminar la columna **income** ya que es la columna que contiene la clase que se pretende predecir:

In [5]:
x = data[['age','workclass','education','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country']]

Los atributos cuyos valores son categorías ('workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'), deben de transformarse a valores numéricos para poder ser utilizados como entradas en los modelos de scikit-learn.

**PREGUNTA: ¿Por qué no es apropiado transformar un atributo de categoría en simples índices numéricos?**

**RESPUESTA: **
El problema de asignarle un índice numérico está en que implícitamente se le asigna un orden. Para ciertos atributos en ciertos casos no estaría mal asignarle un orden como por ejemplo el atributo **Mes**, pero el orden no es laro cuando hablamos del atributo **País**. 

El problema de asignarle orden a los atributos categoriales se puede ver en los clasificadores como por ejemplo en KNN, donde al utilizar la función distancia con el atributo País y sus valores [Uruguay,Argentina,Brasil] y su respectiva codifiación [0,1,2], hace que Uruguay esté mas cerca de Argentina y mas lejos de Uruguay, cuando en realidad tal vez no existía tal relación.
Otros algoritmos, como el id3, no presentan este problema, por lo que pueden trabajar con atributos categoriales sin ningún problema.

Utilice las clases *LabelEncoder* y *OneHotEncoder* del paquete *preprocessing* de *sklearn* para transformar los atributos de categorías en atributos numéricos. Guarde los datos de entrada en una matriz **X**.

In [6]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

categorical_columns = set(['workclass','education','marital-status','occupation','relationship','race','sex','native-country'])
categorical_indices = []

X = pd.DataFrame()
le = LabelEncoder()
unique_categorical_values = 0
for index,attr in enumerate(x):
    if attr in categorical_columns:
        categorical_indices.append(index)
        unique_categorical_values += len(x[attr].unique())
        le.fit(x[attr])
        X[attr] = le.transform(x[attr])
    else:
        X[attr] = pd.Series(x[attr])
print unique_categorical_values
ohe = OneHotEncoder(categorical_features=categorical_indices)
print X.shape
ohe.fit(X)
X = ohe.transform(X)
print X.shape

99
(5000, 12)
(5000, 103)


**PREGUNTA: ¿Cuántos y cuáles son los nuevos atributos del dataset?**

**RESPUESTA:** A partir del shape obtenido antes de aplicarle la transformación de OneHotEncoder, la cantidad de atributos era 12, luego pasó a ser 103, por lo que hay 91 nuevos atributos. Esto se explica porque OneHotEncoder se encarga de agregar un atributo por cada posible valor de los atributos categoriales. Como se desprende de la variable **unique_categorical_values**, estos son 99, mas los 4 atributos continuos son 103 atributos.

Entrene un clasificador, aplique los pasos que crea necesarios para mejorar su performance y evalúe dicho clasificador con validación cruzada (precision, recall, f-score).

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn import feature_selection
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

ley = LabelEncoder()
ley.fit(y)
encoded_y = ley.transform(y)

count_feats = range(1,104,1)
k_neigh = range(1,30,2)
parameters = {'feat__k' : count_feats, 'clf__n_neighbors': k_neigh}
pipe = Pipeline([('feat', feature_selection.SelectKBest(feature_selection.chi2)),
        ('clf', KNeighborsClassifier())])

knn_clf = GridSearchCV(pipe, parameters, n_jobs=-1).fit(X, encoded_y)
print knn_clf.best_params_
print knn_clf.best_score_
predicted = knn_clf.predict(X)

print (metrics.classification_report(encoded_y, predicted, target_names=['<=50K','>50K']))

{'feat__k': 21, 'clf__n_neighbors': 5}
0.829
             precision    recall  f1-score   support

      <=50K       0.89      0.95      0.92      3779
       >50K       0.81      0.62      0.70      1221

avg / total       0.87      0.87      0.87      5000



Como el vector **y** no es numérico, lo primero que se hace es transofmarlo utilizando LabelEncoder. No fue necesario utilizar OneHotEncoder ya que el atributo es binario y además hay un claro orden entre los valores (menor ó mayor que cincuenta mil). 

Luego se hizo una prueba de hiper parámetros en la selección de atributos sobre la cantidad de los mismos y por otro lado en la cantidad de vecinos del clasificador knn. Esta se hizo utilizando Cross Validation con el KFold por defecto de 3. 

Las mejores configuraciones fueron:
n_neighbors=5 y k = 21.

Se logró una precisión y un recall de 87%, por lo tanto f1-score también fue de 87%.